In [2]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

ImportError: Unable to import required dependencies:
numpy: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.7 from "C:\Users\TEEGO\anaconda3\envs\PythonData\python.exe"
  * The NumPy version is: "1.18.5"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: DLL load failed: The specified module could not be found.


In [ ]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

In [ ]:
# Generate summary statistics
df_market_data.describe()

In [ ]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

---

### Prepare the Data

In [ ]:
# get column names

df_market_data.columns

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
market_scaled_data = StandardScaler().fit_transform(df_market_data)

In [ ]:
market_data_scaled_df = pd.DataFrame(
    market_scaled_data,
    columns=['price_change_percentage_24h', 'price_change_percentage_7d',
       'price_change_percentage_14d', 'price_change_percentage_30d',
       'price_change_percentage_60d', 'price_change_percentage_200d',
       'price_change_percentage_1y']
)
# Copy the crypto names from the original data
market_data_scaled_df["coin_id"] = df_market_data.index

# Set the coinid column as index
market_data_scaled_df = market_data_scaled_df.set_index("coin_id")

# Display sample data
market_data_scaled_df.head(10)

---

### Find the Best Value for k Using the Original Data.

In [ ]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 11))

In [ ]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(df_market_data)
    inertia.append(k_model.inertia_)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_curve_data = {"k": k, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve

df_elbow = pd.DataFrame(elbow_curve_data)
df_elbow

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** Elbow Curve: The best value for `k` 4. (k = 4) 4 clusters 

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [ ]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=1)

In [ ]:
# Fit the K-Means model using the scaled data
model.fit(df_market_data)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled data
market_segs = model.predict(df_market_data)

# Print the resulting array of cluster values.
print(market_segs)

In [ ]:
# Create a copy of the DataFrame
market_predictions = df_market_data.copy()

In [ ]:
# Add a new column to the DataFrame with the predicted clusters

market_predictions["Market Clusters"] = market_segs

# Display sample data

market_predictions.head()


In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
market_predictions.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="Market Clusters",
    hover_cols=["coin_id"]
)

---

### Optimize Clusters with Principal Component Analysis.

In [ ]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)
pca

In [ ]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
crypto_pca = pca.fit_transform(df_market_data)

# View the first five rows of the DataFrame. 
crypto_pca[:5]

In [ ]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
pca.explained_variance_ratio_

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** variance total is (0.37+0.35+0.18)*100 = 90%. 

In [ ]:
# Creating a DataFrame with the PCA data
crypto_pca_df = pd.DataFrame(
    crypto_pca,
    columns=["PCA1", "PCA2", "PCA3"]
)
# Copy the crypto names from the original data
crypto_pca_df["coin_id"] = df_market_data.index

# Set the coinid column as index
crypto_pca_df = crypto_pca_df.set_index("coin_id")

# Display sample data
crypto_pca_df.head()

---

### Find the Best Value for k Using the PCA Data

In [ ]:
# Create a list with the number of k-values from 1 to 11
pca_k = list(range(1, 11))

In [ ]:
# Create an empy list to store the inertia values
pca_inertia = []


# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i in pca_k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(crypto_pca_df)
    pca_inertia.append(k_model.inertia_)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
pca_elbow_data = {"k": pca_k, "inertia": pca_inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_pca_elbow = pd.DataFrame(pca_elbow_data)
df_pca_elbow


In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
df_pca_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve with PCA Method", 
    xticks=pca_k
)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** k = 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [ ]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=1)

In [ ]:
# Fit the K-Means model using the PCA data
model.fit(crypto_pca_df)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the PCA data
market_pca_segs = model.predict(crypto_pca_df)

# Print the resulting array of cluster values.
market_pca_segs

In [ ]:
# Create a copy of the DataFrame with the PCA data
crypto_pca_predictions_df = crypto_pca_df.copy()

# Add a new column to the DataFrame with the predicted clusters
crypto_pca_predictions_df["New Predicted Clusters"] = market_pca_segs

# Display sample data
crypto_pca_predictions_df.head()


In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
crypto_pca_predictions_df.hvplot.scatter(
    x="PCA1",
    y="PCA2",
    by="New Predicted Clusters",
    hover_cols=["coin_id"], 
    marker=["hex", "square", "cross", "inverted_triangle"]
)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [ ]:
# Composite plot to contrast the Elbow curves
# Create the plot_1
plot_1 = df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k,
    label="Original Features"
)

# Create plot_2
plot_2 = df_pca_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=pca_k,
    label="With PCA Method"
)

plot_1 + plot_2







In [ ]:
# Composite plot to combined two plots and contrast the clusters
plot_1 + plot_2

# Create plot_1
plot_1 = df_market_predictions.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="Market Clusters",
    hover_cols=["coin_id"],
    label="Original"
)

# Create plot_2
plot_2 = crypto_pca_predictions_df.hvplot.scatter(
    x="PCA1",
    y="PCA2",
    by="New Predicted Clusters",
    hover_cols=["coin_id"], 
    marker=["hex", "square", "cross", "inverted_triangle"],
    label="Post PCA Method"
)


# Combine the plots
combined_plot = plot_1 + plot_2

# Display the composite graph

combined_plot

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** The clusters are relatively the same but the more data seems to overcomplicate the patterns. 
  
  